In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install surprise

    100% |████████████████████████████████| 3.3MB 9.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


# **Content Based Recommender System**

---
I will build two Content Based Recommenders based on:

1. Movie Overviews and Taglines
2. Movie Cast, Crew, Keywords and Genre



In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate, KNNBasic

import warnings; warnings.simplefilter('ignore')

In [0]:
md = pd. read_csv('drive/My Drive/Movies_TMDB/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [0]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [0]:
links_small = pd.read_csv('drive/My Drive/Movies_TMDB/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [0]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')

In [0]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 24)

In [0]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [0]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [0]:
tfidf_matrix.shape

(9099, 268124)

# **Cosine Similarity**

---


I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

**cosine(x,y)=x.y⊺/||x||.||y|| **

Since I have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, I will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

```




In [0]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [0]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [0]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

###

Now we can get top recommendations for the movies.

###


In [0]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [0]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

###

We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, this is not of much use to most people as it doesn't take into considerations very important features such as **cast, crew, director and genre**, which determine the rating and the popularity of a movie. Someone who liked The Dark Knight probably likes it more because of **Christopher Nolan** and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, now I am going to use much more suggestive metadata than Overview and Tagline. In the next subsection, I will build a more sophisticated recommender that takes genre, keywords, cast and crew into consideration.


###

# **Metadata Based Recommender**

In [0]:
credits = pd.read_csv('drive/My Drive/Movies_TMDB/credits.csv')
keywords = pd.read_csv('drive/My Drive/Movies_TMDB/keywords.csv')

In [0]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [0]:
md.shape

(45463, 24)

In [0]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [0]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 27)

###

We now have our cast, crew, genres and credits, all in one dataframe. 

1. **Crew**: From the crew, we will only pick the director as our feature since the others don't contribute that much to the feel of the movie.

2. **Cast**:  Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list.

###

In [0]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))


In [0]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [0]:
smd['director'] = smd['crew'].apply(get_director)

In [0]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [0]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

**###**

These are steps I follow in the preparation of my genres and credits data:

1. **Strip Spaces and Convert to Lowercase** from all our features. This way, our engine will not confuse between Johnny Depp and Johnny Galecki.
  

2.** Mention Director 3 times** to give it more weight relative to the entire cast.

**###**

In [0]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x,x])

###

**Keywords**
We will do a small amount of pre-processing of our keywords before putting them to any use. As a first step, we calculate the frequenct counts of every keyword that appears in the dataset.

###

In [0]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [0]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

###

Keywords occur in frequencies ranging from 1 to 610. We do not have any use for keywords that occur only once. Therefore, these can be safely removed. Finally, we will convert every word to its stem so that words such as Dogs and Dog are considered the same.

###

In [0]:
s = s[s > 1]

In [0]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [0]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [0]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [0]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [0]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [0]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

###

We will reuse the get_recommendations function that we had written earlier. Since our cosine similarity scores have changed, we expect it to give us different (and probably better) results. Let us check for **The Dark Knight** again and see what recommendations I get this time around.

###

In [0]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

###

**Popularity and Ratings**

One thing that we notice about our recommendation system is that it recommends movies regardless of ratings and popularity.

Therefore, we will add a mechanism to remove bad movies and return movies which are popular and have had a good critical response.

I will take the top 25 movies based on similarity scores and calculate the vote of the 60th percentile movie. 

###

I will use **IMDB's weighted rating formula** to construct my list. Mathematically, it is represented as follows:

**Weighted Rating (WR)** =  (v/(v+m).R)+(m/(v+m).C) 
where,

v is the number of votes for the movie

m is the minimum votes required to be listed in the chart

R is the average rating of the movie

C is the mean vote across the whole report

In [0]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.238696808510638

In [0]:
"""m = vote_counts.quantile(0.95)
   m
"""

In [0]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [0]:
"""qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape"""

In [0]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [0]:
#qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [0]:
#qualified = qualified.sort_values('wr', ascending=False).head(250)

**Top Movies**

In [0]:
#qualified.head(15)

In [0]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [0]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,wr
7648,Inception,14075,8,7.919065
8613,Interstellar,11187,8,7.898936
6623,The Prestige,4510,8,7.762198
3381,Memento,4168,8,7.744491
8031,The Dark Knight Rises,9263,7,6.922734
6218,Batman Begins,7511,7,6.905676
1134,Batman Returns,1706,6,5.848168
132,Batman Forever,1529,5,5.051917
9024,Batman v Superman: Dawn of Justice,7189,5,5.013324
1260,Batman & Robin,1447,4,4.281221


# Collaborative Filtering

---


The content based engine suffers from some severe limitations. It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.

Therefore, in this section, I will use a technique called **Collaborative Filtering** to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

I will not be implementing Collaborative Filtering from scratch. Instead, I will use the **Surprise library** that used extremely powerful algorithms like **Singular Value Decomposition (SVD)** ,** K-Nearest Neighbours (KNN)** to minimise **RMSE (Root Mean Square Error)** and give great recommendations.

In [0]:
reader = Reader()

In [0]:
ratings = pd.read_csv('drive/My Drive/Movies_TMDB/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [0]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [0]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9019
MAE:  0.6942
------------
Fold 2
RMSE: 0.8995
MAE:  0.6936
------------
Fold 3
RMSE: 0.8953
MAE:  0.6895
------------
Fold 4
RMSE: 0.9004
MAE:  0.6931
------------
Fold 5
RMSE: 0.8867
MAE:  0.6831
------------
------------
Mean RMSE: 0.8967
Mean MAE : 0.6907
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6942142854138812,
                             0.6936378699779892,
                             0.6894503566788943,
                             0.6931169293569496,
                             0.6831231541308709],
                            'rmse': [0.9018875773222886,
                             0.8994621675402429,
                             0.8952658737960095,
                             0.9003702530489475,
                             0.8867335150742656]})

In [0]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [0]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [0]:
svd.predict(1, 1029, 3)

Prediction(uid=1, iid=1029, r_ui=3, est=2.9024295891137806, details={'was_impossible': False})

In [0]:
knn = KNNBasic()
evaluate(knn, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9647
MAE:  0.7416
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9701
MAE:  0.7462
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9671
MAE:  0.7430
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9744
MAE:  0.7459
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9595
MAE:  0.7406
------------
------------
Mean RMSE: 0.9672
Mean MAE : 0.7434
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.7415931020873899,
                             0.7461684336656618,
                             0.7429553153322702,
                             0.7459149108366451,
                             0.7405818163096164],
                            'rmse': [0.9647165371027947,
                             0.9700940514876377,
                             0.9670897211475563,
                             0.9743512073746022,
                             0.9594991002683045]})

In [0]:
trainset = data.build_full_trainset()
knn.train(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [0]:
knn.predict(1, 1029, 3)

Prediction(uid=1, iid=1029, r_ui=3, est=3.449906718742461, details={'actual_k': 40, 'was_impossible': False})

# Hybrid Recommender

---

In this section, I built a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

**Input** : User ID and the Title of a Movie

**Output** : Similar movies sorted on the basis of expected ratings by                         that particular user.

In [0]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [0]:
id_map = pd.read_csv('drive/My Drive/Movies_TMDB/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [0]:
indices_map = id_map.set_index('id')

In [0]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average',  'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [0]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
8401,Star Trek Into Darkness,4479.0,7.4,54138,3.135939
974,Aliens,3282.0,7.7,679,3.013753
8658,X-Men: Days of Future Past,6155.0,7.5,127585,2.983354
922,The Abyss,822.0,7.1,2756,2.924590
1011,The Terminator,4208.0,7.4,218,2.915419
522,Terminator 2: Judgment Day,4274.0,7.7,280,2.914814
2014,Fantastic Planet,140.0,7.6,16306,2.912532
1668,Return from Witch Mountain,38.0,5.6,14822,2.778753
4017,Hawk the Slayer,13.0,4.5,25628,2.776547
1621,Darby O'Gill and the Little People,35.0,6.7,18887,2.757101


In [0]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,id,est
8401,Star Trek Into Darkness,4479.0,7.4,54138,3.391176
522,Terminator 2: Judgment Day,4274.0,7.7,280,3.264096
1376,Titanic,7770.0,7.5,597,3.147330
344,True Lies,1138.0,6.8,36955,3.063221
1011,The Terminator,4208.0,7.4,218,3.036967
8724,Jupiter Ascending,2816.0,5.2,76757,3.034927
1621,Darby O'Gill and the Little People,35.0,6.7,18887,2.973090
8658,X-Men: Days of Future Past,6155.0,7.5,127585,2.936669
4347,Piranha Part Two: The Spawning,41.0,3.9,31646,2.910802
6084,Beastmaster 2: Through the Portal of Time,17.0,4.6,27549,2.907463


###

We see that for our **hybrid recommender,** we get **different recommendations for different users** although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users.

###